In [2]:
#importing libraries
import os
import sys
import warnings

import numpy as np
import pandas as pd
from mizani.formatters import percent_format
from plotnine import *
from datetime import datetime
import statsmodels.api as sm
import statsmodels.formula.api as smf
from scipy.stats import norm
from IPython.core.display import HTML
from stargazer.stargazer import Stargazer
import statsmodels.nonparametric.kernel_regression as loess

from mizani.transforms import log_trans
from mizani.formatters import percent_format
from mizani.formatters import log_format

warnings.filterwarnings("ignore")

In [8]:
#had to convert 'morg2014.csv' to zip as was too large for github.
data = pd.read_csv('ezyzip.zip')

In [9]:
df=pd.DataFrame(data)
df

,hhid,intmonth,hurespli,minsamp,hrlonglk,hrsample,hrhhid2,serial,hhnum,state,...,ym_file,ym,ch02,ch35,ch613,ch1417,ch05,ihigrdc,docc00,dind02
0,2600310997690,January,1.0,MIS 8,MIS 2-4 Or MIS 6-8 (link To,9100,91001,1,1,63,...,648,633,0,0,0,0,0,18.0,NaN,NaN
1,2600310997690,January,1.0,MIS 8,MIS 2-4 Or MIS 6-8 (link To,9100,91001,1,1,63,...,648,633,0,0,0,0,0,16.0,NaN,NaN
2,2600310997690,January,1.0,MIS 8,MIS 2-4 Or MIS 6-8 (link To,9100,91001,1,1,63,...,648,633,0,0,0,0,0,16.0,Business and financial operations occupations,Administrative and support services
3,41110310970391,January,1.0,MIS 8,MIS 2-4 Or MIS 6-8 (link To,9100,91001,1,1,63,...,648,633,0,0,0,0,0,12.0,NaN,NaN
4,75680310997590,January,1.0,MIS 8,MIS 2-4 Or MIS 6-8 (link To,9100,91001,1,1,63,...,648,633,1,0,1,0,1,14.0,Office and administrative support occupations,"Health care services , except hospitals"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
317051,907086820569600,December,1.0,MIS 8,MIS 2-4 Or MIS 6-8 (link To,9200,92001,1,1,8,...,659,644,0,0,1,0,0,18.0,Management occupations,Public Administration
317052,907086820569600,December,1.0,MIS 8,MIS 2-4 Or MIS 6-8 (link To,9200,92001,1,1,8,...,659,644,0,0,1,0,0,16.0,Management occupations,"Health care services , except hospitals"
317053,947056861569890,December,1.0,MIS 8,MIS 2-4 Or MIS 6-8 (link To,9200,92001,1,1,8,...,659,644,0,0,0,0,0,16.0,NaN,NaN
317054,950868097156649,December,1.0,MIS 8,MIS 2-4 Or MIS 6-8 (link To,9200,92001,1,1,8,...,659,644,0,0,0,0,0,14.0,Office and administrative support occupations,Hospitals


In [10]:
#occupations: mechincal engineers(1460), civil engineers(1360)
data.loc[((data["occ2012"] == 1460) | (data['occ2012'] == 1360)), 'sample'] = 1
#sample == 1
data.loc[data["sample"].isna(), "sample"] = 0 # were NAs input zeros
# sample == 0
data

,hhid,intmonth,hurespli,minsamp,hrlonglk,hrsample,hrhhid2,serial,hhnum,state,...,ym,ch02,ch35,ch613,ch1417,ch05,ihigrdc,docc00,dind02,sample
0,2600310997690,January,1.0,MIS 8,MIS 2-4 Or MIS 6-8 (link To,9100,91001,1,1,63,...,633,0,0,0,0,0,18.0,NaN,NaN,0.0
1,2600310997690,January,1.0,MIS 8,MIS 2-4 Or MIS 6-8 (link To,9100,91001,1,1,63,...,633,0,0,0,0,0,16.0,NaN,NaN,0.0
2,2600310997690,January,1.0,MIS 8,MIS 2-4 Or MIS 6-8 (link To,9100,91001,1,1,63,...,633,0,0,0,0,0,16.0,Business and financial operations occupations,Administrative and support services,0.0
3,41110310970391,January,1.0,MIS 8,MIS 2-4 Or MIS 6-8 (link To,9100,91001,1,1,63,...,633,0,0,0,0,0,12.0,NaN,NaN,0.0
4,75680310997590,January,1.0,MIS 8,MIS 2-4 Or MIS 6-8 (link To,9100,91001,1,1,63,...,633,1,0,1,0,1,14.0,Office and administrative support occupations,"Health care services , except hospitals",0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
317051,907086820569600,December,1.0,MIS 8,MIS 2-4 Or MIS 6-8 (link To,9200,92001,1,1,8,...,644,0,0,1,0,0,18.0,Management occupations,Public Administration,0.0
317052,907086820569600,December,1.0,MIS 8,MIS 2-4 Or MIS 6-8 (link To,9200,92001,1,1,8,...,644,0,0,1,0,0,16.0,Management occupations,"Health care services , except hospitals",0.0
317053,947056861569890,December,1.0,MIS 8,MIS 2-4 Or MIS 6-8 (link To,9200,92001,1,1,8,...,644,0,0,0,0,0,16.0,NaN,NaN,0.0
317054,950868097156649,December,1.0,MIS 8,MIS 2-4 Or MIS 6-8 (link To,9200,92001,1,1,8,...,644,0,0,0,0,0,14.0,Office and administrative support occupations,Hospitals,0.0


In [11]:
#DF with only mechanical and chemical engineers
data = data.loc[data["sample"] == 1].reset_index(drop=True) #keep only sample == 1
data

,hhid,intmonth,hurespli,minsamp,hrlonglk,hrsample,hrhhid2,serial,hhnum,state,...,ym,ch02,ch35,ch613,ch1417,ch05,ihigrdc,docc00,dind02,sample
0,4540720924693,January,1.0,MIS 4,MIS 2-4 Or MIS 6-8 (link To,9200,92001,1,1,94,...,645,0,0,0,0,0,17.0,Architecture and engineering occupations,Professional and Technical services,1.0
1,51250720790591,January,2.0,MIS 4,MIS 2-4 Or MIS 6-8 (link To,9200,92001,1,1,94,...,645,0,0,0,1,0,18.0,Architecture and engineering occupations,Public Administration,1.0
2,70910760948099,January,1.0,MIS 4,MIS 2-4 Or MIS 6-8 (link To,9200,92001,1,1,93,...,645,0,1,0,0,1,16.0,Architecture and engineering occupations,Professional and Technical services,1.0
3,260177093001600,January,1.0,MIS 8,MIS 2-4 Or MIS 6-8 (link To,9100,91001,1,1,93,...,633,0,0,0,0,0,16.0,Architecture and engineering occupations,Petroleum and coal products,1.0
4,310864092903826,January,1.0,MIS 8,MIS 2-4 Or MIS 6-8 (link To,9100,91001,1,1,93,...,633,0,0,0,0,0,16.0,Architecture and engineering occupations,Hospitals,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
864,781317002850290,December,1.0,MIS 8,MIS 2-4 Or MIS 6-8 (link To,101,1011,1,1,5,...,644,0,0,0,0,0,12.0,Architecture and engineering occupations,Professional and Technical services,1.0
865,910043860569190,December,1.0,MIS 8,MIS 2-4 Or MIS 6-8 (link To,9200,92001,1,1,9,...,644,0,0,1,0,0,16.0,Architecture and engineering occupations,Transportation equipment manufacturing,1.0
866,235854026002304,December,1.0,MIS 8,MIS 2-4 Or MIS 6-8 (link To,9200,92001,1,1,9,...,644,0,1,1,0,1,18.0,Architecture and engineering occupations,Transportation equipment manufacturing,1.0
867,64554501500505,December,1.0,MIS 4,MIS 2-4 Or MIS 6-8 (link To,311,3111,1,1,5,...,656,0,0,1,0,0,16.0,Architecture and engineering occupations,Transportation equipment manufacturing,1.0


In [12]:
#value counts for mechanical/chemical engineers occupations
data["sample"].value_counts()

sample
1.0    869
Name: count, dtype: int64

In [13]:
#creating variables; female, wages, lnwages(log of wages), agesq(age squared )
data["female"] = (data["sex"] == 2)
data["w"] = data["earnwke"] / data["uhourse"]
data["lnw"] = np.log(data["w"])
data["agesq"] = np.power(data["age"], 2)
data

,hhid,intmonth,hurespli,minsamp,hrlonglk,hrsample,hrhhid2,serial,hhnum,state,...,ch1417,ch05,ihigrdc,docc00,dind02,sample,female,w,lnw,agesq
0,4540720924693,January,1.0,MIS 4,MIS 2-4 Or MIS 6-8 (link To,9200,92001,1,1,94,...,0,0,17.0,Architecture and engineering occupations,Professional and Technical services,1.0,False,15.375000,2.732743,784
1,51250720790591,January,2.0,MIS 4,MIS 2-4 Or MIS 6-8 (link To,9200,92001,1,1,94,...,1,0,18.0,Architecture and engineering occupations,Public Administration,1.0,False,32.894737,3.493313,2304
2,70910760948099,January,1.0,MIS 4,MIS 2-4 Or MIS 6-8 (link To,9200,92001,1,1,93,...,0,1,16.0,Architecture and engineering occupations,Professional and Technical services,1.0,False,NaN,NaN,2601
3,260177093001600,January,1.0,MIS 8,MIS 2-4 Or MIS 6-8 (link To,9100,91001,1,1,93,...,0,0,16.0,Architecture and engineering occupations,Petroleum and coal products,1.0,False,26.920000,3.292870,2500
4,310864092903826,January,1.0,MIS 8,MIS 2-4 Or MIS 6-8 (link To,9100,91001,1,1,93,...,0,0,16.0,Architecture and engineering occupations,Hospitals,1.0,False,19.230750,2.956511,2809
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
864,781317002850290,December,1.0,MIS 8,MIS 2-4 Or MIS 6-8 (link To,101,1011,1,1,5,...,0,0,12.0,Architecture and engineering occupations,Professional and Technical services,1.0,False,19.000000,2.944439,2704
865,910043860569190,December,1.0,MIS 8,MIS 2-4 Or MIS 6-8 (link To,9200,92001,1,1,9,...,0,0,16.0,Architecture and engineering occupations,Transportation equipment manufacturing,1.0,False,57.675000,4.054824,1936
866,235854026002304,December,1.0,MIS 8,MIS 2-4 Or MIS 6-8 (link To,9200,92001,1,1,9,...,0,1,18.0,Architecture and engineering occupations,Transportation equipment manufacturing,1.0,True,42.788250,3.756264,1296
867,64554501500505,December,1.0,MIS 4,MIS 2-4 Or MIS 6-8 (link To,311,3111,1,1,5,...,0,0,16.0,Architecture and engineering occupations,Transportation equipment manufacturing,1.0,False,30.769111,3.426511,1296


In [14]:
data.shape

(869, 101)